# Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from shutil import copyfile

In [ ]:
import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils
import retsim_cells

In [ ]:
data_utils.make_dir('Figures')

In [ ]:
def create_image(cell, colormap={}, output_filename='temp.png', cone_only=False, d=1, cdia=0.5):
    temp_filename = 'temp.png'
    imsize= 3000
    
    # Render.
    im = cell.init_retsim(
         update=False, print_comps=False,
        d=d, R=True, im_size=imsize, pov_image=temp_filename,
    )
    
    # Plot first image.
    plt.figure(figsize=(6,6))
    plt.imshow(im)
    plt.axis('off')
    plt.show()
    
    # Load pov image file and modify.
    pov_filename_in = cell.retsim_path + temp_filename[:-4] + '.pov'
    pov_filename_out = cell.retsim_path + temp_filename[:-4] + '_new.pov'
    
    print('read\t' + pov_filename_in)
    
    # Read.
    with open(pov_filename_in, "r") as f:
        lines = f.readlines()
    
    # modify width of cylinder.
    if d==6:
        for idx, line in enumerate(lines):
            if 'Yellow' in line:
                lines[idx] = line.replace("1 pigment", str(cdia) + " pigment")
            if '0.5 pigment {Blue}' in line:
                lines[idx] = ''
    
    # modify colors.
    for idx, line in enumerate(lines):
        # Modify GC comp colors
        if '0.5 pigment {Blue}' in line:
            line = line.replace('{Blue}', '{Orange}')
        
        else:
            for old_col, new_col in colormap.items():
                old_col = '{' + old_col + '}'
                new_col = '{' + new_col + '}'
                if old_col in line:
                    line = line.replace(old_col, new_col)
                    break
                
        if "nc_cone" in line: line = line.replace("<0.404,0.404,1>", "<6*0.404,6*0.404,1>")
        if "nc.pov" in line:  line = line.replace("nc.pov", "nc_w.pov")
        
        # Update.
        lines[idx] = line

    
    print('write\t' + pov_filename_out)
            
    # save.
    with open(pov_filename_out, "w+") as f:
        f.writelines(lines)
        
    # Render again.
    im = cell.init_retsim(d=d, R=True, im_size=imsize, update=False, print_comps=False,
                          pov_in_fn=pov_filename_out[:-4], pov_filename=temp_filename)
    
    # Plot new.
    plt.figure(figsize=(6,6))
    plt.imshow(im)
    plt.axis('off')
    plt.show()
    
    # Copy to this folder.
    copyfile(cell.retsim_path+temp_filename,
             'Figures/'+output_filename)

# Cone

In [ ]:
# Create cells.
import importlib
importlib.reload(retsim_cells)

cone = retsim_cells.Cone(
    cone_densfile       = 'dens_cone_optimized_submission2.n',
    nval_file           = 'nval_cone_optimized_submission2.n',
    chanparams_file     = 'chanparams_empty.n',
    expt_file_list      = ['optimize_cones'],
    expt_base_file_list = [os.path.join('..', '..', 'step1a_optimize_cones', 'retsim_files', 'expt_optimize_cones.cc')],
    retsim_path=os.path.abspath(os.path.join('..',  '..', 'neuronc', 'models', 'retsim')) + '/'
)

In [ ]:
cone.create_retsim_expt_file(verbose=False) # Create c++ file.
!(cd {cone.retsim_path} && make) # Compile c++ file.

In [ ]:
cone.init_retsim(verbose=True)

In [ ]:
cone.cone_soma_z = 15

colormap = {'Yellow': 'Red', 'Firebrick': 'Red', 'Green': 'Red', 'Magenta': 'Gray40', 'Red': 'MediumBlue', 'Blue': 'Red'}
output_filename = 'Cone.png'
create_image(cell=cone, colormap=colormap, output_filename=output_filename, d=1)

colormap = {'Blue': 'Black', 'Yellow': 'Gray40'}
output_filename = 'Cone_comps.png'
create_image(cell=cone, colormap=colormap, output_filename=output_filename, d=6, cdia=0.2)

# ON BC

In [ ]:
ON_BC = retsim_cells.CBC(
    bp_type = 'CBC5o',
    expt_file_list      = ['optimize_ON'],
    expt_base_file_list = ['../../step2a_optimize_cbc/retsim_files/expt_CBC_base.cc'],
    retsim_path=os.path.abspath(os.path.join('..',  '..', 'neuronc', 'models', 'retsim')) + '/',
    cone_densfile = 'dens_plot_cones_bp.n',
    bp_densfile = 'dens_plot_bp.n',
    merge_cones=False,
)

In [ ]:
ON_BC.set_rot(myrot=90)

In [ ]:
ON_BC.init_retsim(d=2, verbose=True)

In [ ]:
colormap = {
    'Blue': 'Red',
    'Magenta': 'Gray40',
    'Gray60': 'BlueViolet',
    'Firebrick': 'Navy',
    'Aquamarine': 'MediumBlue',
    'LightBlue': 'SummerSky',
    'White': 'NewMidnightBlue',
    'Yellow': 'NeonBlue'
}
output_filename = 'CBC5.png'
create_image(cell=ON_BC, colormap=colormap, output_filename=output_filename, d=1)

colormap = {'Blue': 'Black', 'Yellow': 'Gray40'}
output_filename = 'CBC5_comps.png'
create_image(cell=ON_BC, colormap=colormap, output_filename=output_filename, d=6, cdia=0.2)

# OFF BC

In [ ]:
OFF_BC = retsim_cells.CBC(
    bp_type = 'CBC3a',
    expt_file_list      = ['optimize_OFF'],
    expt_base_file_list = ['../../step2a_optimize_cbc/retsim_files/expt_CBC_base.cc'],
    retsim_path=os.path.abspath(os.path.join('..',  '..', 'neuronc', 'models', 'retsim')) + '/',
    cone_densfile = 'dens_plot_cones_bp.n',
    bp_densfile = 'dens_plot_bp.n',
    merge_cones=False,
)

In [ ]:
OFF_BC.myrot = 20

In [ ]:
OFF_BC.init_retsim()

In [ ]:
colormap = {
    'Blue': 'Red',
    'Magenta': 'Gray40',
    'Gray60': 'BlueViolet',
    'Firebrick': 'Navy',
    'Aquamarine': 'MediumBlue',
    'LightBlue': 'SummerSky',
    'White': 'NewMidnightBlue',
    'Yellow': 'NeonBlue'
}
output_filename = 'CBC3.png'

create_image(cell=OFF_BC, colormap=colormap, output_filename=output_filename)

# Crop images

In [ ]:
from PIL import Image
from os import listdir
listdir('.')
from matplotlib import pyplot as plt

In [ ]:
im = Image.open('Figures/' + 'Cone.png')

xxyy = (300*3, 0*3, 700*3, 970*3)

# Plot.
plt.imshow(im)
plt.axvline(xxyy[0])
plt.axvline(xxyy[2])
plt.axhline(xxyy[1])
plt.axhline(xxyy[3])

# Crop
im = im.crop(xxyy)

im.save('Figures/' + 'Cone_C.png')

In [ ]:
im = Image.open('Figures/' + 'CBC5.png')

xxyy = (3*270, 3*0, 3*680, 3*970)

# Plot.
plt.imshow(im)
plt.axvline(xxyy[0])
plt.axvline(xxyy[2])
plt.axhline(xxyy[1])
plt.axhline(xxyy[3])

# Crop
im = im.crop(xxyy)

# Crop
im = im.crop()

im.save('Figures/' + 'CBC5_C.png')

In [ ]:
im = Image.open('Figures/' + 'CBC5_comps.png')

xxyy = (3*270, 3*0, 3*680, 3*970)

# Plot.
plt.imshow(im)
plt.axvline(xxyy[0])
plt.axvline(xxyy[2])
plt.axhline(xxyy[1])
plt.axhline(xxyy[3])

# Crop
im = im.crop(xxyy)

im.save('Figures/' + 'CBC5_comps_C.png')

In [ ]:
im = Image.open('Figures/' + 'CBC3.png')

xxyy = (3*300, 3*0, 3*710, 3*970)

# Plot.
plt.imshow(im)
plt.axvline(xxyy[0])
plt.axvline(xxyy[2])
plt.axhline(xxyy[1])
plt.axhline(xxyy[3])

# Crop
im = im.crop(xxyy)

im.save('Figures/' + 'CBC3_C.png')

In [ ]:
im_dark = np.array(im)[:,:,0] < 100 * (np.array(im)[:,:,1] < 100) * (np.array(im)[:,:,2] < 100)
scale_bar_width = np.sum(im_dark, axis=1).max()